# Old ways of using OpenAI API

## via openai python ChatCompletion class

In [47]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

completion=openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role":"system","content":"You are a helpful assistant."},
        {"role":'user',"content":"Hello"}
               
               ]
)

print(completion.choices[0].message)

{
  "role": "assistant",
  "content": "Hi there! How can I assist you today?"
}


## Via CURL command

```bash
curl https://api.openai.com/v1/chat/completions \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -d '{
    "model": "gpt-3.5-turbo",
    "messages": [
      {"role": "system", "content": "You are a helpful assistant."}, 
      {"role": "user", "content": "Hello!"}
      ]
  }'
```

## Via request module

```python
OPENAI_API_KEY=openai.api_key = os.getenv("OPENAI_API_KEY")


import requests

headers  ={

    "Contenet-Type" : "application/json",
    "Authorization" : "Bearer " + OPENAI_API_KEY
}
json_data = {
    'model':'gpt-3.5-turbo',
    'messages': [
        {'role': 'system', 'content': 'You are ahelpful assistant.' },
        {'role': 'user', 'content': 'Hello!' },
    ]
}
response = requests.post(
    "https://api.openai.com/v1/chat/completions",
    headers=headers,
    json=json_data
 )

```


# Weather Chatbot


## Checkout the following example - 

```bash
curl https://api.openai.com/v1/chat/completions -u :$OPENAI_API_KEY -H 'Content-Type: application/json' -d '{
  "model": "gpt-3.5-turbo-0613",
  "messages": [
    {"role": "user", "content": "What is the weather like in Boston?"}
  ],
  "functions": [
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]
          }
        },
        "required": ["location"]
      }
    }
  ]
}'
```

focus on the function keyword and the details under it can be found in a functions docstring. So from the above function description,
you can assume the `get_current_weather` as follows - 

```python
def get_current_weather(location, unit):
    """Get the current weather in a given location

    Args:
        location (str): The city and state, e.g. San Francisco, CA
        unit (str): ["celsius", "fahrenheit"]
    """
    pass
```

In [1]:
import requests
import json
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
GPT_MODEL = 'gpt-3.5-turbo-0613' #new model for function calling abilities



In [2]:
import requests

def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        'Content_Type': 'application/Json',
        'Authorization': 'Bearer ' + OPENAI_API_KEY,
    }
    json_data ={
        "model":model,
        "messages":messages
    }
    if functions is not None:
        json_data.update({"functions":functions})
    if function_call is not None:
        json_data.update({"fuction_call":function_call})

    try:
        response=requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers = headers,
            json = json_data
        )
        return response
    except Exception as e:
        print(f"\n>>>Unable to generate ChatCompletion response")
        print(f"\n>>> Exception: {e}")
        raise e




In [3]:
messages = [
    {'role':'system','content':"you are a helpful assistant"},
    {'role':'user','content':'Hi, there'},
]

chat_response = chat_completion_request(messages=messages)

print(chat_response)
chat_response.json()


<Response [200]>


{'id': 'chatcmpl-7YvE6wmkw8cWkrWVbduapoFIZOHBL',
 'object': 'chat.completion',
 'created': 1688557486,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Hello! How can I assist you today?'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 19, 'completion_tokens': 9, 'total_tokens': 28}}

In [4]:
assistant_message = chat_response.json()
assistant_message['choices'][0]['message']['content']

'Hello! How can I assist you today?'

## Prompting


In [5]:
creator_name = "mach-42"

system_instruction = f"""
You are Weather bot created by {creator_name}, \
You first greet the customer, and you MUST introduce yourself as -
"Hi I am Weather bot created by {creator_name}! How may I assist you today?" \
an automated service to tell Weather infomation \
based on the location provided by the user. \
then collects the location information\
Don't make assumptions about what \
values to plug into functions. \
Ask for clarification if a user request is ambiguous.
You respond in a short, very conversational friendly style. \
"""

In [6]:
messages = [{'role':'system',"content":system_instruction}]
messages

[{'role': 'system',
  'content': '\nYou are Weather bot created by mach-42, You first greet the customer, and you MUST introduce yourself as -\n"Hi I am Weather bot created by mach-42! How may I assist you today?" an automated service to tell Weather infomation based on the location provided by the user. then collects the location informationDon\'t make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.\nYou respond in a short, very conversational friendly style. '}]

In [7]:
functions = [
    {
        "name":"get_current_weather", # Name of the function
        "description":"This weather function will fetch the current weather report",
        'parameters':{
            "type":"object",
            'properties':{ # 1st arguement
                'location':{ #arguement name
                    'type':'string', #arguement type
                    'description':'The city and state, eg- new Delgi, Bengluru' # description of a function with example
                }
            },
            'required':['location'] # required requirement i.e ist arg is required for this function
        }


    }
]

In [8]:
def get_current_weather(location):
    RAPID_API_KEY = os.getenv('RAPID_API_KEY')
    try:

        url = "https://weatherapi-com.p.rapidapi.com/current.json"

        querystring = {"q":location}
        print(f"\n>>>> got the querystring as: {querystring}")


        headers = {
            "X-RapidAPI-Key": RAPID_API_KEY,
            "X-RapidAPI-Host": "weatherapi-com.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers, params=querystring)
        response_json = response.json()
        print(f"\n>>>> got the RAPID API response as: \n{response_json}")
        return response_json

    except Exception as e:
        print("Exception occured in fetching rapid api as : {e}")
        raise e
    

In [9]:
user_message = "Hi there!"
messages.append({"role": "user", "content": user_message})
chat_response = chat_completion_request(messages = messages, functions = functions)
chat_response.json()

{'id': 'chatcmpl-7YvEXuJ9SKEudXmjgiYSGvWsM3VlZ',
 'object': 'chat.completion',
 'created': 1688557513,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Hi! How can I assist you today?'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 170, 'completion_tokens': 10, 'total_tokens': 180}}

### Messge Extraction

In [10]:
messages=[] #Empty the messages list to avoid previous stored data
messages.append({'role':'system',"content":system_instruction})
user_message = "Hi, there."
messages.append({'role': 'user', 'content': user_message})
    
functions = [
    {
        "name":"get_current_weather", # Name of the function
        "description":"This weather function will fetch the current weather report",
        'parameters':{
            "type":"object",
            'properties':{ # 1st arguement
                'location':{ #arguement name
                    'type':'string', #arguement type
                    'description':'The city and state, eg- new Delgi, Bengluru' # description of a function with example
                }
            },
            'required':['location'] # required requirement i.e ist arg is required for this function
        }
    }
]

# Calling the chatgpt api response by passing the message and function
chat_reasponse = chat_completion_request(messages,functions)

# Extracting the message
assistant_message = chat_reasponse.json()['choices'][0]['message']
print(f"\nAssistant Message: \n{assistant_message}") 



Assistant Message: 
{'role': 'assistant', 'content': 'Hi there! How may I assist you today? I am Weather bot created by mach-42!'}


In [11]:
messages.append(assistant_message)
messages

[{'role': 'system',
  'content': '\nYou are Weather bot created by mach-42, You first greet the customer, and you MUST introduce yourself as -\n"Hi I am Weather bot created by mach-42! How may I assist you today?" an automated service to tell Weather infomation based on the location provided by the user. then collects the location informationDon\'t make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.\nYou respond in a short, very conversational friendly style. '},
 {'role': 'user', 'content': 'Hi, there.'},
 {'role': 'assistant',
  'content': 'Hi there! How may I assist you today? I am Weather bot created by mach-42!'}]

In [12]:
user_message = "I want to know the weather at Bengaluru."
messages.append({'role':'user','content':user_message})
chat_reasponse = chat_completion_request(messages,functions)


print(f"\nComplete Chat Response: \n{chat_reasponse.json()}")
assistant_message=chat_reasponse.json()['choices'][0]['message']
print(f"\nAssistant Message: \n{assistant_message}")




Complete Chat Response: 
{'id': 'chatcmpl-7YvEx13e6Dtq2glpgOgcn2yMmXY9F', 'object': 'chat.completion', 'created': 1688557539, 'model': 'gpt-3.5-turbo-0613', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': None, 'function_call': {'name': 'get_current_weather', 'arguments': '{\n  "location": "Bengaluru"\n}'}}, 'finish_reason': 'function_call'}], 'usage': {'prompt_tokens': 209, 'completion_tokens': 19, 'total_tokens': 228}}

Assistant Message: 
{'role': 'assistant', 'content': None, 'function_call': {'name': 'get_current_weather', 'arguments': '{\n  "location": "Bengaluru"\n}'}}


In [13]:
chat_reasponse.json()

{'id': 'chatcmpl-7YvEx13e6Dtq2glpgOgcn2yMmXY9F',
 'object': 'chat.completion',
 'created': 1688557539,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'function_call': {'name': 'get_current_weather',
     'arguments': '{\n  "location": "Bengaluru"\n}'}},
   'finish_reason': 'function_call'}],
 'usage': {'prompt_tokens': 209, 'completion_tokens': 19, 'total_tokens': 228}}

In [14]:
messages

[{'role': 'system',
  'content': '\nYou are Weather bot created by mach-42, You first greet the customer, and you MUST introduce yourself as -\n"Hi I am Weather bot created by mach-42! How may I assist you today?" an automated service to tell Weather infomation based on the location provided by the user. then collects the location informationDon\'t make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.\nYou respond in a short, very conversational friendly style. '},
 {'role': 'user', 'content': 'Hi, there.'},
 {'role': 'assistant',
  'content': 'Hi there! How may I assist you today? I am Weather bot created by mach-42!'},
 {'role': 'user', 'content': 'I want to know the weather at Bengaluru.'}]

In [15]:
assistant_message = chat_reasponse.json()['choices'][0]['message']
assistant_message

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'get_current_weather',
  'arguments': '{\n  "location": "Bengaluru"\n}'}}

In [16]:
import json

def execute_function_call(assistant_message):
    if assistant_message.get('function_call').get('name') == "get_current_weather":
        location = json.loads(assistant_message.get('function_call').get('arguments'))['location']
        results = get_current_weather(location)
    else:
        results = f"Error: function {assistant_message['function_call']['name']} dosent exist."
    return results



In [17]:
results=execute_function_call(assistant_message)
content = json.dumps(results)
content


>>>> got the querystring as: {'q': 'Bengaluru'}

>>>> got the RAPID API response as: 
{'location': {'name': 'Bengaluru', 'region': 'Karnataka', 'country': 'India', 'lat': 12.98, 'lon': 77.58, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1688557549, 'localtime': '2023-07-05 17:15'}, 'current': {'last_updated_epoch': 1688556600, 'last_updated': '2023-07-05 17:00', 'temp_c': 26.0, 'temp_f': 78.8, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 17.4, 'wind_kph': 28.1, 'wind_degree': 250, 'wind_dir': 'WSW', 'pressure_mb': 1007.0, 'pressure_in': 29.74, 'precip_mm': 1.9, 'precip_in': 0.07, 'humidity': 70, 'cloud': 75, 'feelslike_c': 29.2, 'feelslike_f': 84.6, 'vis_km': 8.0, 'vis_miles': 4.0, 'uv': 5.0, 'gust_mph': 23.9, 'gust_kph': 38.5}}


'{"location": {"name": "Bengaluru", "region": "Karnataka", "country": "India", "lat": 12.98, "lon": 77.58, "tz_id": "Asia/Kolkata", "localtime_epoch": 1688557549, "localtime": "2023-07-05 17:15"}, "current": {"last_updated_epoch": 1688556600, "last_updated": "2023-07-05 17:00", "temp_c": 26.0, "temp_f": 78.8, "is_day": 1, "condition": {"text": "Partly cloudy", "icon": "//cdn.weatherapi.com/weather/64x64/day/116.png", "code": 1003}, "wind_mph": 17.4, "wind_kph": 28.1, "wind_degree": 250, "wind_dir": "WSW", "pressure_mb": 1007.0, "pressure_in": 29.74, "precip_mm": 1.9, "precip_in": 0.07, "humidity": 70, "cloud": 75, "feelslike_c": 29.2, "feelslike_f": 84.6, "vis_km": 8.0, "vis_miles": 4.0, "uv": 5.0, "gust_mph": 23.9, "gust_kph": 38.5}}'

In [18]:
def get_natural_response(content):
    convert_prompt = f'convert this resilts from weather api to a natural english sentensce: {content}'
    messages.append({'role':'user','content':convert_prompt})
    convert_prompt_response = chat_completion_request(messages)
    print(f"Received Message: {convert_prompt_response.json()}")
    new_assistant_message = convert_prompt_response.json()["choices"][0]["message"]
    messages.append(new_assistant_message)
    content = new_assistant_message['content']
    print(f"natural response: \n{content}")
    return content

content=get_natural_response(content)


Received Message: {'id': 'chatcmpl-7YvFB1AhmzHqiTy81F1rwHRAcOP3i', 'object': 'chat.completion', 'created': 1688557553, 'model': 'gpt-3.5-turbo-0613', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'The current weather in Bengaluru, Karnataka, India is partly cloudy. The temperature is 26.0°C (78.8°F) and it feels like 29.2°C (84.6°F) due to the 70% humidity. The wind is blowing from the west-southwest at a speed of 28.1 km/h (17.4 mph) with gusts up to 38.5 km/h (23.9 mph). The visibility is 8.0 km (4.0 miles) and the UV index is 5.0.'}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 499, 'completion_tokens': 118, 'total_tokens': 617}}
natural response: 
The current weather in Bengaluru, Karnataka, India is partly cloudy. The temperature is 26.0°C (78.8°F) and it feels like 29.2°C (84.6°F) due to the 70% humidity. The wind is blowing from the west-southwest at a speed of 28.1 km/h (17.4 mph) with gusts up to 38.5 km/h (23.9 mph). The visibility is 8.0 km (4